In [2]:
import os
import re
import json
from tqdm import tqdm
import traceback
import time

def read_abis(abi_path):
    abis = {}
    
    with open(abi_path, 'r', encoding='utf-8') as f:
        data:dict[str, dict[str, dict]] = json.load(f)

    for filename, contracts in data['contracts'].items():
        if filename not in abis:
            abis[filename] = {}

        for contract, details in contracts.items():
            if contract not in abis[filename]:
                abis[filename][contract] = {}
                
            for abi_item in details['abi']:
                
                abi_type = abi_item.get('type', None)
                abi_name = abi_item.get('name', None)
                abi_inputs = abi_item.get('inputs', None)
                
                if abi_type not in abis[filename]:
                    abis[filename][contract][abi_type] = {}
                
                
                if abi_type == 'constructor':
                    abi_name = 'constructor'
                
                if abi_type == 'fallback':
                    abi_name = 'fallback'
                
                if abi_name:
                    abis[filename][contract][abi_type][abi_name] = {}
                
                if abi_inputs:
                    for input_item in abi_inputs:
                        input_type = input_item.get('internalType', None)
                        if input_type and "contract" in input_type:
                            input_type = input_type.replace("contract ", "")
                        if not input_type:
                            input_type = input_item.get('type', None)
                        input_name = input_item.get('name', None)
                        
                        if input_type not in abis[filename][contract][abi_type][abi_name]:

                            abis[filename][contract][abi_type][abi_name][input_type] = []
                        
                        if input_name:
                            abis[filename][contract][abi_type][abi_name][input_type].append(input_name)
    
    return abis

def check_sol_files(directory):
    """Check for Solidity files in the given directory."""
    return [filename for filename in os.listdir(directory) if filename.endswith('.sol')]

def handle_error(e, p, filename=None):
    """Handle errors and write traceback to a file."""
    with open(os.path.join("/home/lxm/solidity/error", p + ".log"), "w") as file:
        file.write(traceback.format_exc())

def delete_comments(allstr):
    allstrs = allstr.split("\n")
    mark = 1
    newstr = ""
    for str in allstrs:
        strs = str.split("\"")
        for i in range(len(strs)):
            if mark == 0:
                if strs[i].find("*/") !=-1:
                    ss = strs[i].split("*/")
                    if len(ss) >= 1:
                        newstr += ss[1]
                    mark = 1
                    continue
                else:
                    continue
            if i % 2 == 0 and mark == 1:
                if strs[i].find("//") !=-1:
                    ss = strs[i].split("//")
                    newstr += ss[0]
                    break
                if strs[i].find("/*") !=-1:
                    ss = strs[i].split("/*")
                    newstr += ss[0]
                    if strs[i].find("*/") !=-1:
                        ss2 = ss[1].split("*/")
                        newstr += ss2[1]
                    else:
                        mark = 0
                    continue
            newstr += strs[i]
            if i != len(strs)-1 : newstr += "\""
        newstr += "\n"
    return newstr

def get_contents(input_file_path):
    # pattern_pragma = r'pragma\s+solidity\s+(\^?>?=?\d+\.\d+\.\d+);'
    pattern_pragma = r'pragma\s+solidity\s+(.*);'
    pattern_import = r"import\s+[\'\"](.+?)[\'\"]\;"
    pattern_from = r"import\s*{.*}\s*from\s*[\'\"](.+?)[\'\"]\;"
    
    with open(input_file_path, 'r') as f:
        data = f.read()
    data = re.sub(pattern_import, '', data)
    data = re.sub(pattern_pragma, '', data)
    data = re.sub(pattern_from, '', data)
    data = delete_comments(data)
    data = "\n".join([i for i in data.split("\n") if i.strip() != ""])
    
    data += '\n'
    return data

def file_import(filepath, G:dict = None, original_dict:dict = None, delete_dict:dict = None):
    if G is None:
        G = {}
    if original_dict is None:
        original_dict = {}
    if delete_dict is None:
        delete_dict = {}
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = f.read()
        pattern_import = r"import\s+[\'\"](.+?)[\'\"]\;"
        pattern_from = r"import\s*{.*}\s*from\s*[\'\"](.+?)[\'\"]\;"
        
        file_name = os.path.basename(filepath)
        if file_name not in G:
            G[file_name] = []
        if file_name not in original_dict:
            matches = re.findall(pattern_import + '|' + pattern_from, data)
            need_add_import = []
            for match in matches:
                if match[0]:
                    matchname = match[0]
                else:
                    matchname = match[1]

                matchpath = os.path.join(os.path.dirname(filepath), matchname[2:])
                if os.path.exists(matchpath):
                    file_import(matchpath, G, original_dict, delete_dict)
                if matchname not in G[file_name]:
                    G[file_name].append(matchname)
                if match[1] and matchname not in need_add_import:
                    need_add_import.append(matchname[2:])

                
            original_dict[file_name] = get_contents(filepath)
            delete_dict[file_name] = get_contents(filepath)

            if need_add_import:
                tmp:str = ""
                for name in need_add_import:
                    tmp = tmp + delete_dict[name]
                    del delete_dict[name]
                delete_dict[file_name] = tmp + delete_dict[file_name]

    except Exception as e:
        pass
    
    return G, original_dict, delete_dict
 


In [3]:
filename = "GnosisSafeProxy.sol"
filepath = "/home/lxm/solidity/solidity/new/codes/0x3cE9Bb52894E2d4Bc3B659B4F7a35f7556cB9FdD_GnosisSafeProxy/GnosisSafeProxy.sol"

abis = read_abis("/home/lxm/solidity/solidity/new/jsons/0x3cE9Bb52894E2d4Bc3B659B4F7a35f7556cB9FdD_GnosisSafeProxy_type1.json")

G, original_dict, delete_dict = file_import(filepath)

# scopes = find_scopes(filepath)
    

In [4]:
for k, v in delete_dict.items():
    print(k)
    print(v)

GnosisSafeProxy.sol
interface IProxy {
    function masterCopy() external view returns (address);
}
contract GnosisSafeProxy {
    address internal singleton;
    constructor(address _singleton) {
        require(_singleton != address(0), "Invalid singleton address provided");
        singleton = _singleton;
    }
    fallback() external payable {
        assembly {
            let _singleton := and(sload(0), 0xffffffffffffffffffffffffffffffffffffffff)
            if eq(calldataload(0), 0xa619486e00000000000000000000000000000000000000000000000000000000) {
                mstore(0, _singleton)
                return(0, 0x20)
            }
            calldatacopy(0, 0, calldatasize())
            let success := delegatecall(gas(), _singleton, 0, calldatasize(), 0, 0)
            returndatacopy(0, 0, returndatasize())
            if eq(success, 0) {
                revert(0, returndatasize())
            }
            return(0, returndatasize())
        }
    }
}
contract GnosisSafeProxy

In [5]:
print(abis)

{'integration_GnosisSafeProxy.sol': {'GnosisSafeProxy': {'constructor': {'constructor': {'address': ['_singleton']}}, 'fallback': {'fallback': {}}}, 'GnosisSafeProxyFactory': {'event': {'ProxyCreation': {'GnosisSafeProxy': ['proxy'], 'address': ['singleton']}}, 'function': {'proxyRuntimeCode': {}}}, 'IProxy': {'function': {'masterCopy': {}}}, 'IProxyCreationCallback': {'function': {'proxyCreated': {'GnosisSafeProxy': ['proxy'], 'address': ['_singleton'], 'bytes': ['initializer'], 'uint256': ['saltNonce']}}}}}


In [8]:
   
def get_parameters(parameters):
    parameters = parameters.split(',')
    parameters = [parameter.strip() for parameter in parameters]
    result = {}
    # pattern = r"import\s+[\'\"](.+?)[\'\"]\;"
    # pattern_1 = r"\s*(address|string|bool)\s*"
    # pattern_2 = r"\s*(byte|uint)\s*"
    
    for parameter in parameters:
        parameter = parameter.split(' ')
        parameter_type = parameter[0]
        parameter_name = parameter[-1]
        if parameter_type not in result:
            result[parameter_type] = []
        result[parameter_type].append(parameter_name)
    return result

def extract_functions(filepath, extracted=None):
    def find_scopes(filename):
        with open(filename, 'r') as f:
            contract_content = f.read()
        
        scopes = {'interface': {}, 'contract': {}, 'library': {}, 'error': {}}
        current_scope = None
        current_name = None
        lines = contract_content.split('\n')
        stack = []
        for line in lines:
            # print(line)
            # print(stack)
            interface_match = re.match(r'^\s*interface\s+(\w+)\s*{', line)
            interface_is_match = re.match(r'^\s*interface\s+(\w+)\s+(is).*{', line)
            contract_match = re.match(r'^(abstract)*\s*contract\s+(\w+)\s*{', line)
            contract_is_match = re.match(r'^(abstract)*\s*contract\s+(\w+)\s+(is).*{', line)
            library_match = re.match(r'^\s*library\s+(\w+)\s*{', line)
            error_match = re.match(r'^(error)\s+(\w+)\s*\((.*?)\)\s*', line)
            if error_match:
                # if '{' in line:
                #     stack.append('{')
                current_scope = 'error'
                current_name = error_match.group(2)
                scopes[current_scope][current_name] = {}
                parameters = error_match.group(3)
                parameters = get_parameters(parameters)
                scopes[current_scope][current_name] = parameters
                current_scope = None
                current_name = None
                continue

            if interface_is_match:
                # print(contract_is_match)
                if '{' in line:
                    stack.append('{')
                # print(stack)
                current_scope = 'contract'
                current_name = interface_is_match.group(1)
                # print(current_name)
                scopes[current_scope][current_name] = []
                continue
            if interface_match:
                stack.append('{')
                # print(stack)
                current_scope = 'interface'
                current_name = interface_match.group(1)
                scopes[current_scope][current_name] = []
                continue
            if contract_is_match:
                # print(contract_is_match)
                if '{' in line:
                    stack.append('{')
                # print(stack)
                current_scope = 'contract'
                current_name = contract_is_match.group(2)
                # print(current_name)
                scopes[current_scope][current_name] = []
                continue
            if contract_match:
                # print(contract_match)
                stack.append('{')
                # print(stack)
                current_scope = 'contract'
                current_name = contract_match.group(2)
                scopes[current_scope][current_name] = []
                continue
            if library_match:
                stack.append('{')
                # print(stack)
                current_scope = 'library'
                current_name = library_match.group(1)
                scopes[current_scope][current_name] = []
                continue
            
            if '{' in line:
            # if re.match(r'*{*', line):
                stack.append('{')
                # print(stack)
                scopes[current_scope][current_name].append(line)
                continue
                
            if re.match(r'^\s*}\s*$', line):
                stack.pop()
                # print(stack)
                if len(stack) == 0:
                    current_scope = None
                    current_name = None
                else:
                    scopes[current_scope][current_name].append(line)
                continue
                
            if current_scope and current_name:
                scopes[current_scope][current_name].append(line)
        return scopes
    events_functions = find_scopes(filepath)
    filename = os.path.basename(filepath)
    extracted[filename] = {}
    error_flag = False
    for contract_scope, items in events_functions.items():
        if contract_scope == 'error':
            if error_flag:
                continue
            extracted[filename]['error'] = events_functions['error']
            error_flag = True
            continue
        for contract_name, code in items.items():
            extracted[filename][contract_name] = {}
            stack = []
            # if contract_name == 'error':
            #     continue
            for index in range(len(code)):
                line = code[index]
                match = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = match.group(3)
                    
                    parameters = get_parameters(parameters)
                    
                    extracted[filename][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(constructor|fallback|receive)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(1)
                    parameters = match.group(2)
                    parameters = get_parameters(parameters)
                    extracted[filename][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\(*', line)
                if match:
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\((.*?)\)\s*', parameters).group(3)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
                match = re.match(r'^\s*(constructor|fallback|receive)\s+(\w+)\s*\(*', line)
                if match:
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(constructor|fallback|receive)\s*\((.*?)\)\s*', parameters).group(2)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
    
    return extracted



In [9]:
extracted = {}

filepath = '/home/lxm/solidity/solidity/new/codes/0x3cE9Bb52894E2d4Bc3B659B4F7a35f7556cB9FdD_GnosisSafeProxy/integration_GnosisSafeProxy.sol'

functions_events = extract_functions(filepath, extracted)

In [10]:
for filename, items in functions_events.items():
    print(f"{filename}:")
    for contract_name, functions in items.items():
        print(f"{contract_name}:")
        if contract_name == 'error':
            # print(functions)
            for error_names, parameters in functions.items():
                print(f"  {error_names}:")
                for parameters_type, parameters in parameters.items():
                    print(f"    {parameters_type}")
                    print(parameters)

            continue
        for function_type, functions in functions.items():
            print(f"  {function_type}:")
            for function_name, parameters in functions.items():
                print(f"    {function_name}")
                # print(parameters)
                for parameters_type, parameters_name in parameters.items():
                    print(f"      {parameters_type}: {parameters_name}")
                # parameters = get_parameters(parameters)
                # print(parameters)
            # print()

integration_GnosisSafeProxy.sol:
IProxy:
  function:
    masterCopy
      : ['']
IProxyCreationCallback:
  function:
    proxyCreated
      GnosisSafeProxy: ['proxy']
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
GnosisSafeProxy:
  constructor:
    constructor
      address: ['_singleton']
  fallback:
    fallback
      : ['']
GnosisSafeProxyFactory:
  event:
    ProxyCreation
      GnosisSafeProxy: ['proxy']
      address: ['singleton']
  function:
    createProxy
      address: ['singleton']
      bytes: ['data']
    proxyRuntimeCode
      : ['']
    proxyCreationCode
      : ['']
    deployProxyWithNonce
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
    createProxyWithNonce
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
    createProxyWithCallback
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
      IProxyCreationCallba